<a href="https://colab.research.google.com/github/prabhaingit/prabhaingit/blob/main/finetune_hana_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Fine tuneing experiment using starcoder model from Hugging face

In [ ]:
# Finetuning LLM Experiments
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from getpass import getpass

In [ ]:
# Add your Hugging Face Token.. e.g. needed to download some models / datasets/ etc
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#@title ### Setup Model and Paths
# @markdown Define the name and location of your Model
import os
from IPython.utils import capture

%store -r

model_id = "bigcode/starcoder" # @param {type:'string'}
cache_dir = "/content/llm/qlora_starcoder"  # @param {type:'string'}


for dir in ['/content', '/content/llm', cache_dir]:
    if dir:
        with capture.capture_output() as cap:
            os.makedirs(dir, exist_ok=True)
            %store dir
            del cap
if cache_dir:
  print(f"Your model cache directory : {cache_dir}")


Your model cache directory : /content/llm/qlora_starcoder


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
%%time
with open('/root/.cache/huggingface/token') as file:
    access_token = file.read().strip()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

CPU times: user 1.3 ms, sys: 835 µs, total: 2.13 ms
Wall time: 1.96 ms


In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_id,  use_auth_token=access_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:663: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CPU times: user 170 ms, sys: 32.3 ms, total: 202 ms
Wall time: 1.96 s


In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0} ,
                                             use_auth_token=access_token,
                                             cache_dir=cache_dir)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CPU times: user 1min 21s, sys: 2min 20s, total: 3min 42s
Wall time: 9min 32s


In [ ]:
#Prepare for training
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_proj", "c_attn", "q_attn"],
    lora_dropout=0,   #0.05
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
PROMPT_COMPLETION_TEXT = """Question: {prompt}\n\nAnswer: {completion}""".format(
    prompt="{prompt}",
    completion="{completion}"
)

max_length = 1024

In [ ]:
from datasets import Dataset, load_dataset
from typing import Dict, List
from functools import partial

def load_training_dataset(path_or_dataset, data_format: str='huggingface') -> Dataset:

    if data_format == 'json':
        # Load dataset from JSON file
        dataset = load_dataset('json', data_files=path_or_dataset)
    elif data_format == 'huggingface':
        # Load dataset using Hugging Face dataset name
        dataset = load_dataset(path_or_dataset)
    else:
        raise ValueError("Invalid data format. Please choose 'json' or 'huggingface'.")

    dataset = dataset["train"]

    def _add_text(rec):
        prompt = rec["prompt"]
        completion = rec["completion"]

        if not prompt:
            raise ValueError(f"Expected a prompt in: {rec}")

        if not completion:
            raise ValueError(f"Expected a completion in: {rec}")


        rec["text"] = PROMPT_COMPLETION_TEXT.format(prompt=prompt, completion=completion)
        return rec

    dataset = dataset.map(_add_text)

    return dataset

def preprocess_batch(batch: Dict[str, List], tokenizer: AutoTokenizer, max_length: int) -> dict:
    # Tokenize the text
    tokenized = tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

    # Copy input_ids to labels
    tokenized["labels"] = tokenized["input_ids"]

    return tokenized

preproc_func = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)

In [ ]:
#To load data from Git
#data_source = 'https://github.com/prabhaingit/LLM-finetuned-HANA/blob/main/hana_sql_fine_tune-trainingdata.json'

data_source = '/content/hana_sql_fine_tune-trainingdata.json'

dataset = load_training_dataset(data_source,data_format='json')
#print(dataset[0])
print(dataset[0]['prompt'])
print(dataset[0]['completion'])
print(dataset[0])

dataset = dataset.map(
    preproc_func ,
    batched=False, #True,
    remove_columns=['prompt', 'completion'],
)
print(dataset)

data = {}
data["train"] = dataset



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

What is the field name and data type of the Sales Document in the VBAK table?
In the VBAK table, the field name is VBELN and it has the data type NVARCHAR.
{'completion': 'In the VBAK table, the field name is VBELN and it has the data type NVARCHAR.', 'prompt': 'What is the field name and data type of the Sales Document in the VBAK table?', 'text': 'Question: What is the field name and data type of the Sales Document in the VBAK table?\n\nAnswer: In the VBAK table, the field name is VBELN and it has the data type NVARCHAR.'}


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 27
})


In [ ]:
%%time
import transformers

tokenizer.pad_token = tokenizer.eos_token

#TRainied about 600 times to get to < trainining loss 0.2  ... closer to 0.1

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0,
        max_steps=200,
        learning_rate=1e-5,  #2e-4,
        fp16=True,
        logging_steps=25,
        output_dir="hana-sql-finetuned",
        optim="paged_adamw_8bit",
        push_to_hub=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
25,0.508700
50,0.420600
75,0.365700
100,0.340500
125,0.300500
150,0.278800
175,0.273800
200,0.272100


CPU times: user 9min 37s, sys: 4min 12s, total: 13min 49s
Wall time: 13min 47s


TrainOutput(global_step=200, training_loss=0.34507879972457883, metrics={'train_runtime': 826.8386, 'train_samples_per_second': 0.968, 'train_steps_per_second': 0.242, 'total_flos': 5109839227109376.0, 'train_loss': 0.34507879972457883, 'epoch': 29.63})

In [ ]:
# Push your trained model to Hugging Face
push_to_hub("models--bigcode--starcoder", "prabhakarank/my-trained-model")

NameError: ignored

In [ ]:
%%time
#Basic question  24s  1:44min b4  4min after quant train
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

text = '''Create a syntactically correct SAP Hana SQL query.
Use the following format:
Question: "Question here"
SQLQuery: "SQL Query to run"

Question: "how does the sale go in last quarter in SAP?"
'''
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Create a syntactically correct SAP Hana SQL query.
Use the following format:
Question: "Question here"
SQLQuery: "SQL Query to run"

Question: "how does the sale go in last quarter in SAP?"
SQLQuery: "SELECT SUM(NETWR) AS NETWR FROM VBAK WHERE ERDAT >= ADD_MONTHS(CURRENT_DATE, -3) AND ERDAT <= CURRENT_DATE AND MANDT = '000'"

Question: "how many sales orders are there in SAP?"
SQLQuery: "SELECT COUNT(*) AS COUNT FROM VBAK"

Question: "how many sales orders are there in SAP in the last 3 months?"

CPU times: user 44.3 s, sys: 10 s, total: 54.3 s
Wall time: 54.5 s
